# Хроматическое число R^4

### 1. Описание алгоритма

Алгоритм:

- строим решетку (grid) и соответствующее ей разбиение Вороного (vor4)
- Ищем все многогранники центрального региона и его диаметр
- составляем список граней центрального многогранника (list_faces)
- перебираю различные подрешетки для каждой ищу приведенный базис, нахожу длину минимального вектора и минимальное расстояние между векторами и их комбинациями:
- Ищем точку s (середина расстояния между точкой (0, 0, 0, 0) и центром ближайшего многогранника
- Нахожу расстояние от точки s до центрального многоранника
- Умножаю это расстояние на 2 и получаю расстояние между многранниками (центральным и ближайшим к нему многогранником из подрешетки)
- составляем списки точек s , расстояния между многогранниками и подрешеток


- grid - изначальная решетка
- central - список вершин центрального многогранника
- coords4 - список координат центров многогранников Вороного
- centr_index - индекс центрального региона (в coords4)
- vor4.vertices - список координат вершин разбиенния Вороного
- vor4.regions - список индексов координат вершин многогранников разбиения Вороного (vor4.vertices)
- vor4.ridge_vertices - координаты вершин 3-мерных граней разбиенния Вороного через их индексы в vor4.vertices
- vor4.ridge_points - список пар центров многогранников Вороного между которыми есть ребро 2х многогранников через индексы в coords4 (по сути - соседние многогранники)

- edge_central_coords - список списков вершин каждой 3-мерной грани центрального многогранника
- faces_3d - список индексов (в vor4.vertices)  координат каждой 3-мерной грани 
- faces_2d - список индексов (в vor4.vertices)  координат каждой 2-мерной грани 
- edges - список индексов (в vor4.vertices)  координат каждого ребра
- list_faces - список 3х мерных граней центрального многогранника с разбивкой каждой 3х мерной грани на 2-мерные


- neighbor_coordinates - список координат центров многогранников, граничащих с центральным (по сути - вектора нормали к 3-мерным граням центрального многогранника) - координаты
 - list_neigh_points_ind - список координат центров многогранников, граничащих с центральным (по сути - вектора нормали к 3-мерным граням центрального многогранника) - индексы
- list_ridge_edge - список граней, в том же порядке, что и соседние точки (list_neigh_points) или что и соответствующие вектора нормали к данным граням
- v_norm - список нормированных векторов нормали (в том же порядке, что и изначальные точки(вектора) нормали)
- vertex_to_faces = собой список списков 3D-граней для каждой вершины (где индексы граней берутся из edge_central_coords)
- point_face_norm - точки, принадлежащие грани и лежащие на векторе нормали (середина вектора, соединяющего центры)
- polyhedrons - данные класса центрального многогранника: грани(2d и 3d), ребра, вектора нормали, центры 

### 2. Подключение библиотек

In [1]:
import numpy as np
from scipy.spatial import Voronoi
from scipy.spatial import distance
from sympy import symbols, Eq, solve
import math
import itertools
import copy
from scipy.spatial import Delaunay
import pandas as pd
import sys
from numpy import linalg as la
from itertools import *
from itertools import product
from copy import deepcopy
from collections import defaultdict

from itertools import combinations
from math import prod
from sympy import factorint

import networkx as nx
from scipy.spatial import ConvexHull

from scipy.spatial import distance
from fpylll import IntegerMatrix, LLL

import time

from VoronoiPolyhedra import *

from FindSubGrids import *

from SPoint import *

from FindDistForDet import *

/Users/konstantinglushkov/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 3. Построение многогранника Вороного

In [2]:

grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)
'''
grid = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype = float)

grid = np.array([
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1],
    [0.5, 0.5, 0.5, 0.5]
], dtype = float)


grid = np.array([
    [1, 1, 0, 0],
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1]
], dtype = float)



grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)
'''
vor4 = VoronoiPolyhedra(grid)

vor4.findCentral()
vor4.fillData()
vor4.findFaces3d()
vor4.findFaces2d()
vor4.findEdges()
vor4.findNeigh()

# диаметр многогранника (ищем максимальное расстояние между вершинами) - 2
max_len = round(2 * distance.euclidean(np.array([0, 0, 0, 0]), vor4.central[1]), 10)
min_d = 1

vor4.find_2d_subfaces()

vor4.count_v_norm()

vor4.createPolyhedrons()

vor4.get_vertex_faces_3d()

суммарное расстояние = 24.0 индекс центрального региона = 393


### 4. Триангуляция Делоне

In [3]:
# строим триангуляцию

delaunay = Delaunay(vor4.central)

# Получаем все вершины симплексов
simplices_vertices = delaunay.simplices

# Уникальные вершины
unique_vertices = np.unique(simplices_vertices)

# Проверяем, что все точки из central используются
if len(unique_vertices) == len(vor4.central):
    print('Все точки используются в триангуляции.')
else:
    print('Не все точки используются в триангуляции.')

# Выпуклая оболочка исходных точек
original_hull = ConvexHull(vor4.central)

# Все точки триангуляции
triangulated_points = vor4.central[delaunay.simplices.flatten()]

# Выпуклая оболочка точек триангуляции
triangulated_hull = ConvexHull(triangulated_points)

# Сравнение объемов выпуклых оболочек
is_convex_hull_correct = np.isclose(original_hull.volume, triangulated_hull.volume)
print('Выпуклость триангуляции сохранена:', is_convex_hull_correct)

Все точки используются в триангуляции.
Выпуклость триангуляции сохранена: True


### 5. Поиск подрешеток

In [4]:
det_dist, det_center, det_mat = find_dist_for_det(range(49, 51), 2, max_len, grid, vor4, delaunay)

49
1 7 7
1.0801234497346432 [1 0 2 3]
1.0801234497346432 [-3  0  1  2]
1.0801234497346432 [1 2 0 3]
1.0801234497346432 [-3  1  0  2]
1.0801234497346432 [ 1 -2  0  3]
1.0801234497346432 [1 0 3 2]
1.0801234497346432 [1 0 2 3]
1.0801234497346432 [-1  3  0  2]
1.0801234497346432 [-3  0  1  2]
1.0801234497346432 [ 1  3  0 -2]
1.0801234497346432 [-3 -1  0  2]
1.0801234497346432 [1 0 3 2]
1 1 49
50
2 5 5
1 2 25
1 5 10
1 1 50


### Старое

In [4]:
limits = 2 #целое число, задающее границы генерации коэффициентов, используемых при построении точек решетки 

# ключ: координаты центра многогранника, значение - расстояние от s до центр. многогранника
centers_dist = dict() 


norm_length = 1 / max_len

det_dist = dict() # словарь где ключ - определитель, значение - расстояние
det_center = dict() # словарь где ключ - определитель, значение - координаты точки s
det_mat = dict()

for det in range(49, 51):

    print(det)
    
    list_all_factorizations = compute_factorizations(det)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)
    
    # среди этих расстояний надо найти максимальное для каждой матрицы mat
    mat_dist = dict()
    mat_center = dict()
    list_mats = []
    index = 0
    
    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], float)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        print(max_num_col1, max_num_col2, max_num_col3)

        for indices in product(range(max_num_col3), range(max_num_col3), range(max_num_col2),
                               range(max_num_col3), range(max_num_col2), range(max_num_col1)):
            i1, i2, i3, i4, i5, i6 = indices
            mat[2][3] = i1
            mat[1][3] = i2
            mat[1][2] = i3
            mat[0][3] = i4
            mat[0][2] = i5
            mat[0][1] = i6

            #print('mat', mat)
            
            sub_grid = np.dot(mat, grid)
            #basis = LLL(sub_grid)
            sub_grid_int = (sub_grid).astype(int)
            basis = LLL.reduction(IntegerMatrix.from_matrix(sub_grid_int.tolist()))
            rows, cols = basis.nrows, basis.ncols
            sub_grid_LLL = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)])

            centers = lattice_points_no_central_symmetry(sub_grid_LLL, limits,\
                                                         3, max_len)
            min_dist_mat = 2 * max_len

            if len(centers) == 1 and np.array_equal(centers[0], np.array([0, 0, 0, 0])):
                continue

            for center in centers:
                center_key = tuple(center)

                if center_key in centers_dist:
                    dist = centers_dist[center_key]
                else:
                    s = 0.5 * center
                    dist, coords, ind = dist_to_s(vor4.polyhedrons, s, vor4, delaunay, max_len)
                    # добавляем новое значение в centers_dist
                    centers_dist[center_key] = dist

                if dist < min_dist_mat:
                    min_dist_mat = dist
                    min_center = center

                if dist > min_dist_mat + max_len: continue

                if min_dist_mat < 1: break



            if min_dist_mat < 1: continue

            print(min_dist_mat, min_center)

            # сохраняем значения для mat
            list_mats.append(mat)

            mat_dist[index] = min_dist_mat
            mat_center[index] = min_center
            index += 1
                                
    # находим максимальный среди минимальных элементов по расстоянию для данного определителя
    if mat_dist:
        max_pair = max(mat_dist.items(), key=lambda item: item[1])
        det_dist[det] = max_pair[1] # ключ - определитель, значение - расстояние
        det_center[det] = mat_center[max_pair[0]] # ключ - определитель, значение - координаты центра
        det_mat[det] = list_mats[max_pair[0]] # ключ - определитель, значение - матрица mat

49
1 7 7
1 1 49
50
2 5 5
1 2 25
1 5 10
1 1 50


### 8. LLL-базис (функция)

In [ ]:
def gram_schmidt(basis):
    n = len(basis)
    b_star = [np.array(basis[i], dtype=float) for i in range(n)]
    mu = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i):
            mu[i,j] = np.dot(basis[i], b_star[j]) / np.dot(b_star[j], b_star[j])
            b_star[i] -= mu[i,j] * b_star[j]
        mu[i,i] = np.dot(b_star[i], b_star[i])  
    
    return b_star, mu


def size_reduction(basis, b_star, mu, k, l):
    '''Выполняет Size-Reduction шаг.'''
    q = round(mu[k, l])
    basis[k] -= q * basis[l].copy()
    mu[k, l] -= q
    for j in range(l):
        mu[k, j] -= q * mu[l, j]


def interchange(basis, mu, b_star, beta, k):
    '''Выполняет Interchange шаг.'''
    # Меняем местами векторы
    basis[k], basis[k - 1] = basis[k - 1].copy(), basis[k].copy()
    
    # Обновляем mu
    for j in range(k - 1):
        mu[k, j], mu[k - 1, j] = mu[k - 1, j], mu[k, j]
    
    mu_k_km1 = mu[k, k - 1]
    beta_k = beta[k] - mu_k_km1 ** 2 * beta[k - 1]
    beta_km1 = beta[k - 1]
    
    
    # Обновляем mu
    mu[k, k - 1] = mu_k_km1 * beta_km1 / beta_k
    b = b_star[k-1]
    b_star[k-1] = b_star[k] + mu_k_km1 * b
    b_star[k] = (beta[k]/beta_k) * b - mu_k_km1 * b_star[k]
    
    # Обновляем beta
    beta[k] = beta[k - 1] * beta[k] / beta_k
    beta[k - 1] = beta_k
    
    
    for i in range(k + 1, len(basis)):
        temp = mu[i, k]
        mu[i, k] = mu[i, k - 1] - mu_k_km1 * temp
        mu[i, k - 1] = temp + mu[k, k-1] * mu[i, k]


def LLL_basis(basis, delta=0.75):
    n = len(basis)
    basis = [np.array(vec, dtype=float) for vec in basis]
    b_star, mu = gram_schmidt(basis)
    beta = [np.dot(b_star[i], b_star[i]) for i in range(n)]
    
    k = 1
    while k < n:
        # Size Reduction
        for l in range(k-1, -1, -1):
            if abs(mu[k,l]) > 0.5:
                q = round(mu[k,l])
                basis[k] -= q * basis[l]
                # Обновляем mu
                mu[k,l] -= q
                for j in range(l):
                    mu[k,j] -= q * mu[l,j]
                # Пересчитываем Грама-Шмидта после изменения базиса
                b_star, mu = gram_schmidt(basis)
                beta = [np.dot(b_star[i], b_star[i]) for i in range(n)]
        
        # Проверка условия Ловаса
        if beta[k] < (delta - mu[k,k-1]**2) * beta[k-1]:
            # Interchange
            basis[k], basis[k-1] = basis[k-1].copy(), basis[k].copy()
            # Пересчитываем всё после обмена
            b_star, mu = gram_schmidt(basis)
            beta = [np.dot(b_star[i], b_star[i]) for i in range(n)]
            k = max(1, k-1)
        else:
            k += 1
    
    return np.array(basis)

In [ ]:
# поиск приведенного базиса через библиотеку fpylll

list_mat = []
list_lll = []
list_s1 = []
list_dist = []

norm_length = 1 / max_len

for det_mat in range(74, 75):

    print(det_mat)

    list_all_factorizations = compute_factorizations(det_mat)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)

    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], int)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        #max_num_col0 = diag_el[0]
        #print(max_num_col1, max_num_col2, max_num_col3)
        
        num_iterations = diag_el[1] * diag_el[2] * diag_el[2] * diag_el[3] * diag_el[3] * diag_el[3]
        iter = 0
        
        print("start diag factors:", diag_el[1], diag_el[2], diag_el[3])
        print("num iterations:", num_iterations)

        for i1 in range(max_num_col3):
            mat[2][3] = i1
            print(i1)
            for i2 in range(max_num_col3):
                mat[1][3] = i2
                for i3 in range(max_num_col2):
                    mat[1][2] = i3
                    for i4 in range(max_num_col3):
                        mat[0][3] = i4
                        for i5 in range(max_num_col2):
                            mat[0][2] = i5
                            for i6 in range(max_num_col1):
                                mat[0][1] = i6
                                
                                iter += 1
                                if iter % 10 == 0:
                                    print("[", int(10000 * iter / num_iterations) / 100, "% ]", "   iter:", iter)
                                
                                sub_grid = np.dot(mat, grid)
                                sub_grid_int = (sub_grid).astype(int)
                                basis = LLL.reduction(IntegerMatrix.from_matrix(sub_grid_int.tolist()))
                                rows, cols = basis.nrows, basis.ncols
                                list_of_lists = [[basis[i, j] for j in range(cols)] for i in range(rows)]
                                s1 = s_point(np.array(list_of_lists), vor4)
                                dist_s1 = dist_to_s(vor4.polyhedrons, s1, vor4, delaunay, max_len)
                               
                                if dist_s1[0] >= 1:                                   
                                    print(dist_s1, s1, mat)
                                    list_dist.append(dist_s1)
                                    list_mat.append(mat.copy())
                                    list_lll.append(list_of_lists.copy())
                                    list_s1.append(s1.copy())

In [ ]:
# поиск приведенного базиса через библиотеку fpylll

list_mat = []
list_lll = []
list_s1 = []
list_dist = []

norm_length = 1 / max_len

for det_mat in range(74, 75):

    print(det_mat)

    list_all_factorizations = compute_factorizations(det_mat)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)

    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], int)
        
        num_iterations = diag_el[1] * diag_el[2] * diag_el[2] * diag_el[3] * diag_el[3] * diag_el[3]
        iter = 0
        
        print("start diag factors:", diag_el[1], diag_el[2], diag_el[3])
        print("num iterations:", num_iterations)

        for i1 in range(diag_el[3]):
            mat[2][3] = i1
            for i2 in range(diag_el[3]):
                mat[1][3] = i2
                for i3 in range(diag_el[2]):
                    mat[1][2] = i3
                    for i4 in range(diag_el[3]):
                        mat[0][3] = i4
                        for i5 in range(diag_el[2]):
                            mat[0][2] = i5
                            for i6 in range(diag_el[1]):
                                mat[0][1] = i6
                                
                                iter += 1
                                if iter % 10 == 0:
                                    print("[", int(10000 * iter / num_iterations) / 100, "% ]", "   iter:", iter)
                                
                                sub_grid = np.dot(mat, grid)
                                sub_grid_int = (sub_grid).astype(int)
                                basis = LLL.reduction(IntegerMatrix.from_matrix(sub_grid_int.tolist()))
                                
                                rows, cols = basis.nrows, basis.ncols
                                list_of_lists = [[basis[i, j] for j in range(cols)] for i in range(rows)]
                                
                                points = s_points(np.array(list_of_lists))
                                                              
                                min_point = points[0]
                                min_dist_s = dist_to_s(vor4.polyhedrons, min_point, vor4, delaunay, max_len)[0]
                                
                                for point in points:
                                    dist_s = dist_to_s(vor4.polyhedrons, point, vor4, delaunay, max_len)[0]
                                    if dist_s < min_dist_s:
                                        min_dist_s = dist_s
                                        min_point = point

                                if min_dist_s >= 1:                                   
                                    print(min_dist_s, min_point)
                                    print(mat)
                                    list_dist.append(min_dist_s)
                                    list_mat.append(mat.copy())
                                    list_lll.append(list_of_lists.copy())
                                    list_s1.append(min_point.copy())

### Эксперименты

In [ ]:
from scipy.spatial import distance

def s_point_2(sub_grid):
    digits = np.array(list(product(range(-2, 2), repeat=4)))  # Все комбинации
    #print(digits, 'digits')
    coords4_1 = np.dot(digits, sub_grid)  # Все точки подрешётки
    
    # Удаляем нулевую точку
    coords4_1 = coords4_1[~np.all(coords4_1 == 0, axis=1)]
    
    # Минимальное расстояние между точками
    if len(coords4_1) >= 2:
        dist_matrix = distance.pdist(coords4_1)
        min_dist = np.min(dist_matrix)
        if min_dist <= 2:
            return np.array([0, 0, 0, 0])
    
    # Поиск общей точки с coords4
    common_coord = np.array([x for x in coords4_1 if x.tolist() in vor4.coords4])
    if len(common_coord) == 0:
        return np.array([0, 0, 0, 0])
    
    # Ближайшая к центру точка
    dist_to_center = np.linalg.norm(common_coord, axis=1)
    #print(dist_to_center)
    #print('common_coord', common_coord)
    #print('dist_to_center', dist_to_center)
    closest_idx = np.argmin(dist_to_center)
    return common_coord * 0.5

In [ ]:
mat = list_mat[0]
mat = np.array([[ 1,  0,  0, 25],
 [ 0,  1,  0, 57],
 [ 0,  0,  1, 45],
 [ 0,  0,  0, 60]])
sub_grid = np.dot(mat, grid)
sub_grid_int = (sub_grid).astype(int)
basis = LLL.reduction(IntegerMatrix.from_matrix(sub_grid_int.tolist()))
rows, cols = basis.nrows, basis.ncols
list_of_lists = [[basis[i, j] for j in range(cols)] for i in range(rows)]
#basis = LLL(sub_grid)
s1 = s_point_2(np.array(list_of_lists))

In [ ]:
ld = []
for i in s1:
    ld.append(dist_to_s(vor4.polyhedrons, i, vor4, delaunay, max_len))
min([sublist[0] for sublist in ld if sublist])

In [ ]:
# поиск приведенного базиса через библиотеку fpylll

list_mat_4 = []
list_lll_4 = []
list_s1_4 = []
list_dist_4 = []

norm_length = 1 / max_len

for det_mat in range(74, 80):

    print(det_mat)

    list_all_factorizations = compute_factorizations(det_mat)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)

    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], int)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        #max_num_col0 = diag_el[0]
        
        num_iterations = diag_el[1] * diag_el[2] * diag_el[2] * diag_el[3] * diag_el[3] * diag_el[3]
        iter = 0
        
        print("start diag factors:", diag_el[1], diag_el[2], diag_el[3])
        print("num iterations:", num_iterations)

        for i1 in range(max_num_col3):
            mat[2][3] = i1
            print(i1)
            for i2 in range(max_num_col3):
                mat[1][3] = i2
                for i3 in range(max_num_col2):
                    mat[1][2] = i3
                    for i4 in range(max_num_col3):
                        mat[0][3] = i4
                        for i5 in range(max_num_col2):
                            mat[0][2] = i5
                            for i6 in range(max_num_col1):
                                mat[0][1] = i6
                                
                                iter += 1
                                if iter % 10 == 0:
                                    print("[", int(10000 * iter / num_iterations) / 100, "% ]", "   iter:", iter)
                                
                                
                                
                                sub_grid = np.dot(mat, grid)
                                sup_grid_lll = make_good(sub_grid)

                                centers = center_points(sup_grid_lll)
                                min_dist = 2 * diam
                                min_center

                                for center in centers:
                                    centers_dist = ...
                                    if centers_dist > min_dist + diam: continue
                                    
                                    if center in dist_from_poly:
                                        dist = dist_from_poly[center]
                                    else:
                                        dist = dist_to_s(...)
                                    
                                    if dist < min_dist:
                                        min_dist = dist
                                        min_center = center
                                    
                                    if min_dist < 1: break
                                    
                                if min_dist >= 1:
                                    print(min_dist, min_center)


In [ ]:
# поиск приведенного базиса через библиотеку fpylll

list_mat_4 = []
list_lll_4 = []
list_s1_4 = []
list_dist_4 = []

norm_length = 1 / max_len

for det_mat in range(64, 74):

    print(det_mat)

    list_all_factorizations = compute_factorizations(det_mat)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)

    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], int)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        #max_num_col0 = diag_el[0]
        print(max_num_col1, max_num_col2, max_num_col3)

        for i1 in range(max_num_col3):
            mat[2][3] = i1
            print(i1)
            for i2 in range(max_num_col3):
                mat[1][3] = i2
                for i3 in range(max_num_col2):
                    mat[1][2] = i3
                    for i4 in range(max_num_col3):
                        mat[0][3] = i4
                        for i5 in range(max_num_col2):
                            mat[0][2] = i5
                            for i6 in range(max_num_col1):
                                mat[0][1] = i6
                                
                                sub_grid = np.dot(mat, grid)
                                sub_grid_int = (sub_grid).astype(int)
                                basis = LLL.reduction(IntegerMatrix.from_matrix(sub_grid_int.tolist()))
                                rows, cols = basis.nrows, basis.ncols
                                list_of_lists = [[basis[i, j] for j in range(cols)] for i in range(rows)]
                                s1 = s_point(np.array(list_of_lists), vor4)
                                dist_s1 = dist_to_s(vor4.polyhedrons, s1, vor4, delaunay, max_len)
                               
                                if dist_s1[0] >= 1:                                   
                                    #print(dist_s1, s1, mat)
                                    s1 = s_point_2(np.array(list_of_lists))
                                    ld = []
                                    for i in s1:
                                        ld.append(dist_to_s(vor4.polyhedrons, i, vor4, delaunay, max_len))
                                    dist_s1 = min([sublist[0] for sublist in ld if sublist])
                                    if dist_s1 >= 1:
                                        print(dist_s1, s1, mat)
                                        list_dist_4.append(dist_s1)
                                        list_mat_4.append(mat.copy())
                                        list_lll_4.append(list_of_lists.copy())
                                        list_s1_4.append(s1.copy())

In [ ]:
digits = np.array(list(product(range(-2, 2), repeat=4)))  # Все комбинации
coords4_1 = np.dot(digits, list_lll[0])
for i in range(len(coords4_1)):
    if all(coords4_1[i] == t):
        print(i, coords4_1[1])
#np.array(list_lll[0][0]) + np.array(list_lll[0][1]) == t

In [ ]:
# поиск приведенного базиса через функцию LLL

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


norm_length = 1 / max_len

for det_mat in range(46, 50):

    print(det_mat)

    list_all_factorizations = compute_factorizations(det_mat)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)

    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], float)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        #max_num_col0 = diag_el[0]
        print(max_num_col1, max_num_col2, max_num_col3)

        for i1 in range(max_num_col3):
            mat[2][3] = i1
            print(i1)
            for i2 in range(max_num_col3):
                mat[1][3] = i2
                for i3 in range(max_num_col2):
                    mat[1][2] = i3
                    for i4 in range(max_num_col3):
                        mat[0][3] = i4
                        for i5 in range(max_num_col2):
                            mat[0][2] = i5
                            for i6 in range(max_num_col1):
                                mat[0][1] = i6
                                
                                sub_grid = np.dot(mat, grid)
                                basis = LLL(sub_grid)
                                
                                s1 = s_point(basis)
                                dist_s1 = dist_to_s(polyhedrons, s1)
                                
                                if dist_s1[0] >= 1:                                   
                                    print(dist_s1, s1, mat)
                                    list_dist.append(dist_s1)
                                    list_mat.append(mat.copy())
                                    list_lll.append(basis.copy())
                                    list_s1.append(s1.copy())

In [ ]:
def lattice_points_no_central_symmetry(basis, limits, r_max):
    """
    Генерирует точки решетки, заданной базисом, без центральносимметричных точек,
    сразу исключая симметричные коэффициенты.
    
    :param limits: Целое число, определяющее диапазон коэффициентов для генерации точек (-limits, limits).
    :param r_max: максимальное расстояние, дальше которого нет смысла рассматривать()
    :return: Список коэффициентов без центральносимметричных точек.  
    """
    
    # Инициализируем список для хранения точек
    points = []
    
    # Генерируем все возможные целочисленные комбинации коэффициентов
    for coeffs in product(range(-limits, limits + 1), repeat=4):
        if all(c == 0 for c in coeffs):  # Пропускаем нулевую точку
            continue
        
        # Проверяем "каноничность" коэффициентов
        # Каноническая форма: первый ненулевой коэффициент должен быть положительным
        first_nonzero_index = next((i for i, c in enumerate(coeffs) if c != 0), None)
        
        if first_nonzero_index is not None and coeffs[first_nonzero_index] < 0:
            continue  # Пропускаем, если первый ненулевой коэффициент отрицательный

        # Вычисляем точку решетки
        point = np.dot(coeffs, basis)
    
        # Предварительная фильтрация: ограничение по максимальной координате
        max_coord = int(np.ceil(r_max))  # Максимально возможное значение координаты
        
        # Проверяем, что координаты точки находятся в допустимых пределах
        if any(abs(c) >= max_coord for c in point):  # Отсеиваем точки, которые заведомо далеки
            continue
        
        points.append(point)

    # Минимальное расстояние между точками
    if len(points) >= 2:
        dist_matrix = distance.pdist(points)
        min_dist = np.min(dist_matrix)

        if min_dist <= max_len:
            points = []
            points.append(np.array([0, 0, 0, 0]))
            return points
    
    return points

In [6]:
limits = 2 #целое число, задающее границы генерации коэффициентов, используемых при построении точек решетки 

# ключ: координаты центра многогранника, значение - расстояние от s до центр. многогранника
centers_dist = dict() 


norm_length = 1 / max_len

det_dist = dict() # словарь где ключ - определитель, значение - расстояние
det_center = dict() # словарь где ключ - определитель, значение - координаты точки s
det_mat = dict()

for det in range(49, 51):

    print(det)
    
    list_all_factorizations = compute_factorizations(det)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)
    
    # среди этих расстояний надо найти максимальное для каждой матрицы mat
    mat_dist = dict()
    mat_center = dict()
    list_mats = []
    index = 0
    
    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], float)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        print(max_num_col1, max_num_col2, max_num_col3)

        for indices in product(range(max_num_col3), range(max_num_col3), range(max_num_col2),
                               range(max_num_col3), range(max_num_col2), range(max_num_col1)):
            i1, i2, i3, i4, i5, i6 = indices
            mat[2][3] = i1
            mat[1][3] = i2
            mat[1][2] = i3
            mat[0][3] = i4
            mat[0][2] = i5
            mat[0][1] = i6

            #print('mat', mat)
            
            sub_grid = np.dot(mat, grid)
            #basis = LLL(sub_grid)
            sub_grid_int = (sub_grid).astype(int)
            basis = LLL.reduction(IntegerMatrix.from_matrix(sub_grid_int.tolist()))
            rows, cols = basis.nrows, basis.ncols
            sub_grid_LLL = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)])

            #print('sub_grid_LLL', sub_grid_LLL)

            #s1 = s_point(np.array(list_of_lists), vor4)
            #dist_s1 = dist_to_s(vor4.polyhedrons, s1, vor4, delaunay, max_len)

            centers = lattice_points_no_central_symmetry(sub_grid_LLL, limits,\
                                                         3, max_len)
            min_dist_mat = 2 * max_len

            if len(centers) == 1 and np.array_equal(centers[0], np.array([0, 0, 0, 0])):
                continue

            for center in centers:
                center_key = tuple(center)

                if center_key in centers_dist:
                    dist = centers_dist[center_key]
                else:
                    s = 0.5 * center
                    dist, coords, ind = dist_to_s(vor4.polyhedrons, s, vor4, delaunay, max_len)
                    # добавляем новое значение в centers_dist
                    centers_dist[center_key] = dist

                if dist < min_dist_mat:
                    min_dist_mat = dist
                    min_center = center

                if dist > min_dist_mat + max_len: continue

                if min_dist_mat < 1: break



            if min_dist_mat < 1: continue

            print(min_dist_mat, min_center)

            # сохраняем значения для mat
            list_mats.append(mat)

            mat_dist[index] = min_dist_mat
            mat_center[index] = min_center
            index += 1
                                
    # находим максимальный среди минимальных элементов по расстоянию для данного определителя
    if mat_dist:
        max_pair = max(mat_dist.items(), key=lambda item: item[1])
        det_dist[det] = max_pair[1] # ключ - определитель, значение - расстояние
        det_center[det] = mat_center[max_pair[0]] # ключ - определитель, значение - координаты центра
        det_mat[det] = list_mats[max_pair[0]] # ключ - определитель, значение - матрица mat

49
1 7 7
1.0801234497346432 [1 0 2 3]
1.0801234497346432 [-3  0  1  2]
1.0801234497346432 [1 2 0 3]
1.0801234497346432 [-3  1  0  2]
1.0801234497346432 [ 1 -2  0  3]
1.0801234497346432 [1 0 3 2]
1.0801234497346432 [1 0 2 3]
1.0801234497346432 [-1  3  0  2]
1.0801234497346432 [-3  0  1  2]
1.0801234497346432 [ 1  3  0 -2]
1.0801234497346432 [-3 -1  0  2]
1.0801234497346432 [1 0 3 2]
1 1 49
50
2 5 5
1 2 25
1 5 10
1 1 50


In [7]:
det_dist, det_center, det_mat

({49: 1.0801234497346432},
 {49: array([1, 0, 2, 3])},
 {49: array([[1., 0., 6., 6.],
         [0., 1., 6., 6.],
         [0., 0., 7., 6.],
         [0., 0., 0., 7.]])})

In [6]:
dist, coords, ind = dist_to_s(vor4.polyhedrons, s, vor4, delaunay, max_len)

UnboundLocalError: cannot access local variable 'coords_proj' where it is not associated with a value